## Topic Modelling 

This notebook contains topic modelling script on the collected data.

Table of Content:
* **1. Import required packages**
* **2. Import files**


### 1. Import required packages

In [24]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis.sklearn

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation as LDA

### 2. Import files

In [2]:
abt_covid = pd.read_pickle('C:/Users/molna/Desktop/Szakdolgozat/adatok/abt_covid_featured.pkl')

print("ABT table has {} rows and {} columns".format(len(abt_covid), len(abt_covid.columns)))

ABT table has 63633 rows and 50 columns


### 2.1 Separate mainstream and dezinformational articles

In [3]:
abt_covid_mainstream = abt_covid[abt_covid["dezinf"] == 0]

In [4]:
abt_covid_dezinf = abt_covid[abt_covid["dezinf"] == 1]

### 3. NMF on dezinformational articles

In [15]:
tfidf_vectorizer_dezinf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf_dezinf = tfidf_vectorizer.fit_transform(abt_covid_dezinf["text_lemmas"].astype(str))
tfidf_feature_names_dezinf = tfidf_vectorizer.get_feature_names()

C:\Users\molna\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
# A Nonnegative Double Singular Value Decomposition metódussal a ritkán előforduló értékeket jobban figyelembe vesszük.
# A l1_ratio egy normalizációs paraméter ami 0 és 1 közötti értéket vehet fel. A 0.5-ös érték a Frobenius norm értékének felel meg.
# Az aplha paraméter egy konstans amivel megszorozzuk a normalizált értékeket. Ha 0, nincs normalizálás.
nmf = NMF(n_components=5, random_state=45, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_dezinf)

C:\Users\molna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,


In [7]:
#Függvény a topikok megjelenítéséhez.
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("\nTopic #%d:" % topic_idx)
        print( " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [17]:
no_top_words = 20
display_topics(nmf, tfidf_feature_names_dezinf, no_top_words)


Topic #0:
vírus koronavírus ember betegség tünet tud beteg járvány világ fertőzés százalék eset nap súlyos idő orvos ír covi kórház teszt

Topic #1:
szám fertőzött fő elhunytak aktív beteg beazonosított fertőzés nő en állampolgár idős haza elhunyt emelkedik gyógyul közül kórház koronavírus magyar

Topic #2:
oltás vakcina oltópont pfizer oltási oltóanyag dolgozó astrazenec védőoltás regisztrált gov moderna háziorvos adag érkezik megkap millió olt beolt hu

Topic #3:
tilalom kijárási továbbra óra dolgozó szabály este intézkedés egészségügyi szociális zárt ingyenes idős kötelező általános maszkviselés érvény ellátás fő tilos

Topic #4:
orbán viktor miniszterelnök kormány gulyás magyarország gergely magyar járvány ország operatív miniszter intézkedés törzs korlátozás döntés védekezés kormányfő szabály hullám


### 4. NMF on mainstream articles

In [18]:
tfidf_vectorizer_main = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
tfidf_main = tfidf_vectorizer.fit_transform(abt_covid_mainstream["text_lemmas"].astype(str))
tfidf_feature_names_main = tfidf_vectorizer.get_feature_names()

C:\Users\molna\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
# A Nonnegative Double Singular Value Decomposition metódussal a ritkán előforduló értékeket jobban figyelembe vesszük.
# A l1_ratio egy normalizációs paraméter ami 0 és 1 közötti értéket vehet fel. A 0.5-ös érték a Frobenius norm értékének felel meg.
# Az aplha paraméter egy konstans amivel megszorozzuk a normalizált értékeket. Ha 0, nincs normalizálás.
nmf = NMF(n_components=5, random_state=45, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_main)

C:\Users\molna\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,


In [11]:
no_top_words = 20
display_topics(nmf, tfidf_feature_names_main, no_top_words)


Topic #0:
szám beteg fertőzött kórház fő fertőzés nő nap koronavírus aktív egészségügyi koronavírusos közül müller járvány óra idős cecília en emelkedik

Topic #1:
fertőzött halálos áldozat szám gyógyult en halott nyilván regisztrál haláleset egyesült hal ra ország igazolt betegség tart eset nyilvántartás covi

Topic #2:
vakcina oltás oltóanyag pfizer adag oltási astrazenec orosz védőoltás szputnyik olt biontech millió moderna magyarország oltópont sinopharm beolt kínai megkap

Topic #3:
csapat játékos bajnokság mérkőzés fradi klub meccs válogatott szövetség labdarúgó liga ferencváros edző szezon bajnok támadó boys young magyar játszik

Topic #4:
magyar kormány százalék milliárd járvány forint európai ország magyarország koronavírus gazdaság orbán tud ember gazdasági cég támogatás viktor helyzet tesz


### 5. LDA on dezinformational articles

In [14]:
# number of topic
number_topics = 5

# Hány szót jelenítsen meg egy topikból:
number_words = 20

In [21]:
# build tha modell and add hyperparameters:
lda_dezinf = LDA(n_components=number_topics, n_jobs=-1, random_state=45)

# fit the LDA modell
lda_dezinf.fit(tfidf_dezinf)

LatentDirichletAllocation(n_components=5, n_jobs=-1, random_state=45)

In [26]:
pickle.dump(lda_dezinf, open('C:/Users/molna/Desktop/Szakdolgozat/lda_dezinf.pk', 'wb'))

In [27]:
display_topics(lda_dezinf, tfidf_feature_names_dezinf, number_words)


Topic #0:
fecó hercegovina boszik koszovó kim montenegró vérrög boncolás macedónia rék timi hanks csontvelősejt korál ricsi öttel bkk lisziewicz tom bajnok

Topic #1:
fokhagyma víz gyömbér méz folt vércsoport tea fogyaszt hatás alkaloida hatóanyag gyógynövény bácsi kannabisz tudasfaj vitamin gyógyszer mézet kever hatású

Topic #2:
koronavírus szám oltás vakcina fertőzött járvány ember beteg vírus tud fertőzés magyarország magyar fő kórház ország nap kormány egészségügyi idős

Topic #3:
nyugdíj pápa havi forint nyugdíjas alapjövedelem asztrológus bd feri emelés megdöbbentő jani egyházfő futár georgica járadék nézőpont hisz nyugellátás visszaépítés

Topic #4:
rotavírus szájer operatőr síszezon sas zivatar elefánt kuncze apukám zápor csapadék berardelli drávaszerdahely intelligenciaszint nozokomiális nicole monoxid dúl cf ereszcsatorna


In [17]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(nmf, small_document_term_matrix,small_count_vectorizer,mds='tsne')


NameError: name 'small_document_term_matrix' is not defined